In [2]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

<ipython-input-2-f898bd0df230>:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
%pip install ultralytics
import ultralytics
from ultralytics import YOLO

In [5]:
#Не забудь переминовать папку в "datasets"
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Z7xJwizM7hpSTsiIk6Dv")
project = rf.workspace("roadsegmentation").project("roadsegmentation-with-damages")
dataset = project.version(1).download("yolov8")

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

KeyboardInterrupt: ignored

In [8]:
model = YOLO('yolov8n-seg.pt')
model.train(data='data.yaml', epochs=40, imgsz=640)
model.val()

Ultralytics YOLOv8.0.173 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=data.yaml, epochs=40, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, i

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fecabf9be20>
fitness: 1.0834682320574869
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)', 'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
maps: array([    0.49989,     0.35254,     0.92898,      1.1598,      1.4707,      1.0905,      1.7925,      1.7679,      1.2603,     0.86044,     0.18874])
names: {0: 'catsEye', 1: 'craks', 2: 'patchs', 3: 'pothole', 4: 'roadAsphalt', 5: 'roadMarking', 6: 'roadPaved', 7: 'roadUnpaved', 8: 'speedBump', 9: 'stormDrain', 10: 'waterPuddle'}
plot: True
results_dict: {'metrics/precision(B)': 0.7648508951132533, 'metrics/recall(B)': 0.7383012406793173, 'metrics/mAP50(B)': 0.7616604717292276, 'metrics/mAP50-95(B)': 0.5415290

In [5]:
model.export()

Ultralytics YOLOv8.0.173 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/segment/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 47, 8400), (1, 32, 160, 160)) (6.5 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 3.9s, saved as 'runs/segment/train/weights/best.torchscript' (12.9 MB)

Export complete (5.9s)
Results saved to /content/runs/segment/train/weights
Predict:         yolo predict task=segment model=runs/segment/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=segment model=runs/segment/train/weights/best.torchscript imgsz=640 data=data.yaml  
Visualize:       https://netron.app


'runs/segment/train/weights/best.torchscript'

In [5]:
model = YOLO('runs/segment/train4/weights_seg_road/best.pt')

In [9]:
from PIL import Image

im1 = Image.open("datasets/test/images/000000013_png.rf.94335264206600ff64283e3d77d7239c.jpg")
results = model.predict(source=im1, save=True)


0: 640x640 5 crakss, 1 patchs, 1 roadAsphalt, 1 waterPuddle, 15.2ms
Speed: 2.0ms preprocess, 15.2ms inference, 26.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/predict3


In [6]:
#VIDEO

# accepts all formats - image/dir/Path/URL/video/PIL/ndarray. 0 for webcam
results = model('drive/MyDrive/register_1.mp4', save=True)  # generator of Results objects



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/969) /content/drive/MyDrive/register_1.mp4: 384x640 1 roadAsphalt, 106.9ms
video 1/1 (2/969) /content/drive/MyDrive/register_1.mp4: 384x640 1 roadAsphalt, 7.6ms
video 1/1 (3/969) /content/drive/MyDrive/register_1.mp4: 384x640 1 roadAsphalt, 9.3ms
video 1/1 (4/969) /content/drive/MyDrive/register_1.mp4: 384x640 1 roadAsphalt, 8.6ms
video 1/1 (5/969) /content/drive/MyDrive/register_1.mp4: 384x640 2 roadAsphalts, 8.0ms
video 1/1 (6/969) /cont